In [32]:
from __future__ import print_function, division
%matplotlib inline
import statsmodels.formula.api as smf  
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import sys
from scipy import stats
from fiona.crs import from_epsg
import shapely

In [33]:
#nyc open data API, you need apptoken to run this cell
from sodapy import Socrata
client = Socrata("data.cityofnewyork.us", os.getenv("apptoken"))

#### 311 Top 5 that have laglon in records

In [34]:
results = client.get("fhrw-4uyv",
                     select = "created_date, complaint_type, latitude, longitude",
                     where="created_date > '2013-01-01T00:00:12.000'",
                     limit=167000)
df = pd.DataFrame.from_records(results)
df.head()

,complaint_type,created_date,latitude,longitude
0,Noise - Residential,2017-08-27T00:17:09.000,40.79305387572195,-73.82362901281846
1,Blocked Driveway,2017-08-26T22:13:39.000,40.635390562953276,-74.02135114295945
2,Noise - Residential,2017-08-26T23:30:45.000,40.676080164805306,-73.96131304275177
3,Noise - Residential,2017-08-27T00:23:22.000,40.847514601225335,-73.90874235667427
4,Noise - Residential,2017-08-27T01:27:01.000,40.761575678058996,-73.92568893447253


In [35]:
df.dropna(inplace=True)
df.count()

complaint_type    151026
created_date      151026
latitude          151026
longitude         151026
dtype: int64

In [36]:
countgroup = df.groupby(by='complaint_type').count()
countgroup.reset_index(inplace=True)
countgroup.sort_values('created_date', ascending=False, inplace=True)

In [37]:
countgroup.head(10)

,complaint_type,created_date,latitude,longitude
58,Graffiti,21065,21065,21065
89,Noise - Residential,12116,12116,12116
42,Electronics Waste,11108,11108,11108
68,Illegal Parking,6762,6762,6762
12,Blocked Driveway,5988,5988,5988
142,UNSANITARY CONDITION,5873,5873,5873
90,Noise - Street/Sidewalk,5267,5267,5267
154,Water System,4266,4266,4266
129,Street Condition,4193,4193,4193
130,Street Light Condition,4160,4160,4160


In [38]:
results1 = client.get("fhrw-4uyv",
                     select = "created_date, complaint_type, latitude, longitude",
                     where="complaint_type='Noise - Residential' and created_date > '2013-01-01T00:00:12.000'",
                     limit=16700000)
dftop1 = pd.DataFrame.from_records(results1)
dftop1.head()

,complaint_type,created_date,latitude,longitude
0,Noise - Residential,2016-01-22T00:00:58.000,40.74639869446456,-73.92373536844984
1,Noise - Residential,2016-01-21T23:48:07.000,40.81975332877146,-73.9522007764116
2,Noise - Residential,2016-01-21T22:05:22.000,40.739598814750124,-73.92204711134542
3,Noise - Residential,2016-01-21T19:30:42.000,40.75426121652461,-73.93054450626512
4,Noise - Residential,2016-01-21T19:16:33.000,40.75426121652461,-73.93054450626512


In [39]:
results2 = client.get("fhrw-4uyv",
                     select = "created_date, complaint_type, latitude, longitude",
                     where="complaint_type='Electronics Waste' and created_date > '2013-01-01T00:00:12.000'",
                     limit=16700000)
dftop2 = pd.DataFrame.from_records(results2)
dftop2.head()

,complaint_type,created_date,latitude,longitude
0,Electronics Waste,2016-10-15T10:30:00.000,40.59233141774145,-74.08440614579655
1,Electronics Waste,2016-10-27T18:01:00.000,40.643758035828505,-74.07631307164182
2,Electronics Waste,2016-12-04T11:20:00.000,40.62490567192611,-74.14132919461646
3,Electronics Waste,2017-01-09T06:35:00.000,40.505848619646116,-74.23950334238465
4,Electronics Waste,2017-01-25T09:21:00.000,40.62401517852025,-74.11256974865324


In [40]:
results3 = client.get("fhrw-4uyv",
                     select = "created_date, complaint_type, latitude, longitude",
                     where="complaint_type='Illegal Parking' and created_date > '2013-01-01T00:00:12.000'",
                     limit=16700000)
dftop3 = pd.DataFrame.from_records(results3)
dftop3.head()

,complaint_type,created_date,latitude,longitude
0,Illegal Parking,2013-09-04T12:27:14.000,NaN,NaN
1,Illegal Parking,2017-08-01T10:08:28.000,40.705092892963336,-73.78349513895796
2,Illegal Parking,2017-08-01T09:36:17.000,40.705296608963046,-73.77104036881622
3,Illegal Parking,2017-08-02T14:20:51.000,40.713428813959176,-73.89895929775781
4,Illegal Parking,2017-08-03T20:24:21.000,40.59279825283227,-73.98837706244852


In [41]:
results4 = client.get("fhrw-4uyv",
                     select = "created_date, complaint_type, latitude, longitude",
                     where="complaint_type='Blocked Driveway' and created_date > '2013-01-01T00:00:12.000'",
                     limit=16700000)
dftop4 = pd.DataFrame.from_records(results4)
dftop4.head()

,complaint_type,created_date,latitude,longitude
0,Blocked Driveway,2016-01-21T22:35:43.000,40.756620735051385,-73.81017893154225
1,Blocked Driveway,2016-01-21T13:33:02.000,40.58981503207579,-73.96380415913065
2,Blocked Driveway,2016-01-21T12:59:42.000,40.63063599020575,-73.89659517677445
3,Blocked Driveway,2016-01-21T15:58:23.000,40.689817779761405,-73.8661524983311
4,Blocked Driveway,2016-01-21T18:55:00.000,40.688659208582926,-73.83622271420703


In [42]:
results5 = client.get("fhrw-4uyv",
                     select = "created_date, complaint_type, latitude, longitude",
                     where="complaint_type='Graffiti' and created_date > '2013-01-01T00:00:12.000'",
                     limit=16700000)
dftop5 = pd.DataFrame.from_records(results5)
dftop5.head()

,complaint_type,created_date,latitude,longitude
0,Graffiti,2017-06-29T07:43:30.000,40.66440998381232,-73.99033265650645
1,Graffiti,2017-07-10T10:57:49.000,40.814665167963135,-73.91883061617878
2,Graffiti,2017-07-10T14:47:37.000,40.69779412789359,-73.93670885584137
3,Graffiti,2017-07-10T12:18:20.000,40.70028544312668,-73.94562168025375
4,Graffiti,2017-07-10T11:49:38.000,40.67845463349782,-73.86872785053247


In [43]:
dftop1.shape

(986762, 4)

In [44]:
dftop2.shape

(15777, 4)

In [45]:
dftoptotal = pd.concat([dftop1, dftop2])
dftoptotal = pd.concat([dftoptotal, dftop3])
dftoptotal = pd.concat([dftoptotal, dftop4])
dftoptotal = pd.concat([dftoptotal, dftop5])
dftoptotal.shape

(2013043, 4)

In [46]:
dftoptotal.dropna(inplace=True)
dftoptotal.count()

complaint_type    2000317
created_date      2000317
latitude          2000317
longitude         2000317
dtype: int64

In [47]:
dftoptotal = dftoptotal.sample(frac=0.01)

In [48]:
# combine lat and lon to one column
dftoptotal['lonlat']=list(zip(dftoptotal.longitude.astype(float), dftoptotal.latitude.astype(float)))
# Create Point Geometry for based on lonlat column
dftoptotal['geometry']=dftoptotal[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))
dftoptotal.head()

,complaint_type,created_date,latitude,longitude,lonlat,geometry
86981,Noise - Residential,2016-06-14T23:35:53.000,40.812149546291366,-73.94408165802088,"(-73.94408165802088, 40.812149546291366)",POINT (-73.94408165802088 40.81214954629137)
166502,Noise - Residential,2016-10-15T17:36:59.000,40.88718311738633,-73.90461560115159,"(-73.90461560115159, 40.88718311738633)",POINT (-73.90461560115159 40.88718311738633)
69367,Noise - Residential,2016-05-21T23:46:12.000,40.669925616391204,-73.89148029496046,"(-73.89148029496046, 40.669925616391204)",POINT (-73.89148029496046 40.6699256163912)
22118,Blocked Driveway,2016-04-03T09:25:29.000,40.84906822070742,-73.89591223550404,"(-73.89591223550404, 40.84906822070742)",POINT (-73.89591223550404 40.84906822070742)
221974,Illegal Parking,2015-09-12T16:47:44.000,40.595950285864376,-73.75410245027925,"(-73.75410245027925, 40.595950285864376)",POINT (-73.75410245027925 40.59595028586438)


In [49]:
dftoptotal = gpd.GeoDataFrame(dftoptotal)
dftoptotal.crs = from_epsg(4326)

map

In [50]:
censusblock = gpd.read_file("nyc_census_blocks/nycb2010.shp")
censusblock.head()

,CB2010,BoroCode,BoroName,CT2010,BCTCB2010,Shape_Leng,Shape_Area,geometry
0,1000,5,Staten Island,000900,50009001000,2508.948115,244589.603884,"POLYGON ((962269.1260375977 173705.5001831055,..."
1,1000,5,Staten Island,002001,50020011000,1345.886422,111006.286826,"POLYGON ((964642.2676391602 161835.2651977539,..."
2,1000,5,Staten Island,002700,50027001000,1703.381172,150406.766022,"POLYGON ((963363.1168212891 167356.3336181641,..."
3,1000,5,Staten Island,004000,50040001000,1511.173743,141296.561149,"POLYGON ((960069.9965820312 161953.6936035156,..."
4,1000,5,Staten Island,006400,50064001000,1978.243852,200784.983607,"POLYGON ((963023.891784668 158246.7521972656, ..."


In [51]:
censusblock.crs = from_epsg(2263)
#censusblock = censusblock.to_crs(epsg=4326)

In [52]:
censusblock.head()

,CB2010,BoroCode,BoroName,CT2010,BCTCB2010,Shape_Leng,Shape_Area,geometry
0,1000,5,Staten Island,000900,50009001000,2508.948115,244589.603884,"POLYGON ((962269.1260375977 173705.5001831055,..."
1,1000,5,Staten Island,002001,50020011000,1345.886422,111006.286826,"POLYGON ((964642.2676391602 161835.2651977539,..."
2,1000,5,Staten Island,002700,50027001000,1703.381172,150406.766022,"POLYGON ((963363.1168212891 167356.3336181641,..."
3,1000,5,Staten Island,004000,50040001000,1511.173743,141296.561149,"POLYGON ((960069.9965820312 161953.6936035156,..."
4,1000,5,Staten Island,006400,50064001000,1978.243852,200784.983607,"POLYGON ((963023.891784668 158246.7521972656, ..."


In [53]:
dftoptotal.crs = from_epsg(4326)
dftoptotal = dftoptotal.to_crs(epsg=2263)
dftoptotal.reset_index(inplace=True)
dftoptotal

,index,complaint_type,created_date,latitude,longitude,lonlat,geometry
0,86981,Noise - Residential,2016-06-14T23:35:53.000,40.812149546291366,-73.94408165802088,"(-73.94408165802088, 40.812149546291366)",POINT (999728.9999907895 235170.0144247717)
1,166502,Noise - Residential,2016-10-15T17:36:59.000,40.88718311738633,-73.90461560115159,"(-73.90461560115159, 40.88718311738633)",POINT (1010623.999984276 262517.014441938)
2,69367,Noise - Residential,2016-05-21T23:46:12.000,40.669925616391204,-73.89148029496046,"(-73.89148029496046, 40.669925616391204)",POINT (1014353.999982167 183367.0143914943)
3,22118,Blocked Driveway,2016-04-03T09:25:29.000,40.84906822070742,-73.89591223550404,"(-73.89591223550404, 40.84906822070742)",POINT (1013046.999982846 248633.0144332564)
4,221974,Illegal Parking,2015-09-12T16:47:44.000,40.595950285864376,-73.75410245027925,"(-73.75410245027925, 40.595950285864376)",POINT (1052538.999959651 156493.0143737737)
5,436485,Illegal Parking,2017-06-16T11:14:06.000,40.785375068218684,-73.80546754314581,"(-73.80546754314581, 40.785375068218684)",POINT (1038120.999967983 225470.01441856)
6,149566,Noise - Residential,2016-09-18T00:40:41.000,40.71315252614694,-73.95089928581771,"(-73.95089928581771, 40.71315252614694)",POINT (997861.9999919296 199101.0144017182)
7,473465,Blocked Driveway,2017-11-15T09:11:50.000,40.67620400511047,-73.90851531109448,"(-73.90851531109448, 40.67620400511047)",POINT (1009625.999984966 185649.0143929826)
8,133318,Blocked Driveway,2017-03-02T22:37:49.000,40.88944047578409,-73.85780099100222,"(-73.85780099100222, 40.88944047578409)",POINT (1023566.999976552 263357.0144424475)
9,70481,Graffiti,2015-07-25T16:10:47.000,40.815338051522204,-73.82705983363964,"(-73.82705983363964, 40.815338051522204)",POINT (1032119.99997152 236374.0144254871)


In [ ]:
dftoptotal

In [54]:
total = gpd.sjoin(
    censusblock, dftoptotal, how='left', op='within')
total.head()

,CB2010,BoroCode,BoroName,CT2010,BCTCB2010,Shape_Leng,Shape_Area,geometry,index_right,index,complaint_type,created_date,latitude,longitude,lonlat
0,1000,5,Staten Island,000900,50009001000,2508.948115,244589.603884,"POLYGON ((962269.1260375977 173705.5001831055,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,5,Staten Island,002001,50020011000,1345.886422,111006.286826,"POLYGON ((964642.2676391602 161835.2651977539,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000,5,Staten Island,002700,50027001000,1703.381172,150406.766022,"POLYGON ((963363.1168212891 167356.3336181641,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000,5,Staten Island,004000,50040001000,1511.173743,141296.561149,"POLYGON ((960069.9965820312 161953.6936035156,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000,5,Staten Island,006400,50064001000,1978.243852,200784.983607,"POLYGON ((963023.891784668 158246.7521972656, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
total.to_csv('top5_since_2013.csv')

#### standing water 311 only

In [26]:
results = client.get("fhrw-4uyv",
                     select = "unique_key, created_date, complaint_type, descriptor, latitude, longitude",
                     where="complaint_type='Standing Water' and created_date > '2013-01-01T00:00:12.000'",
                     limit=16700000, order="created_date")
df = pd.DataFrame.from_records(results)
df.head()

,complaint_type,created_date,descriptor,latitude,longitude,unique_key
0,Standing Water,2013-01-05T13:44:31.000,Swimming Pool - Unmaintained,40.62144994711384,-74.16408066097868,24728741
1,Standing Water,2013-01-07T15:59:43.000,Swimming Pool - Unmaintained,40.62695470252081,-74.17964263529007,24740658
2,Standing Water,2013-02-11T19:54:14.000,Swimming Pool - Unmaintained,40.62695470252081,-74.17964263529007,24978704
3,Standing Water,2013-03-04T10:36:13.000,Swimming Pool - Unmaintained,40.7796595881746,-73.84060984867408,25111897
4,Standing Water,2013-03-09T11:33:39.000,Swimming Pool - Unmaintained,40.577025434643666,-74.12718139854422,25149129


In [27]:
df.complaint_type.unique()

array(['Standing Water'], dtype=object)

In [28]:
df.dropna(inplace=True)
df.count()

complaint_type    14258
created_date      14258
descriptor        14258
latitude          14258
longitude         14258
unique_key        14258
dtype: int64

In [29]:
df.to_csv('standing_water_since_2013.csv')